In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [3]:
data1 = [[ 0.5,  0.7, None, None,  0.3,  0.2,  0.5],
         [ 0.1,  0.2, None, 0.3 , None,  0.6, None],
         [ 0.2, None,  0.8, None, None, None, 0.6 ],
         [0.05, 0.03, None, 0.07,  0.1, None, 0.08],
         [ None, None, 0.01, None, 0.05, None, 0.05]]
data2 = [[500, 1000, 300, 300,  800, 200, 100],
         [600,  500, 200,   0,  400, 300, 150],
         [300,  600,   0,   0,  500, 400, 100],
         [200,  300, 400, 500,  200,   0, 100],
         [  0,  100, 500, 100, 1000, 300,   0],
         [500,  500, 100, 300, 1100, 500,  60]]

PRODS = ['PROD1', 'PROD2', 'PROD3', 'PROD4', 'PROD5', 'PROD6', 'PROD7']
MONTHS = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']
MACHINES = ['grinder', 'v_driller', 'h_driller', 'borer', 'planer']
PREV_MONTH = {'JAN': 'INITIAL', 
              'FEB': 'JAN',
              'MAR': 'FEB',
              'APR': 'MAR',
              'MAY': 'APR',
              'JUN': 'MAY'}

req_proc = pd.DataFrame(data1, columns=PRODS, index=MACHINES)
mark_lim = pd.DataFrame(data2, columns=PRODS, index=MONTHS)
exist_mac = dict(zip(MACHINES, [4, 2, 3, 1, 1]))
req_maint = dict(zip(MACHINES, [2, 2, 3, 1, 1]))
contr_profit = dict(zip(PRODS, [10, 6, 8, 4, 11, 9, 3]))

storage_cost   = 0.5
storage_limit  = 100
shift_length   = 8
n_of_shifts    = 2
days_per_month = 24

In [4]:
model = gp.Model('Factory Planning II')

# add vars
make = model.addVars(PRODS, MONTHS,
                     name='make')
sell = model.addVars(PRODS, MONTHS,
                     name='sell')
store = model.addVars(PRODS, MONTHS+['INITIAL'],
                       name='store')
maint = model.addVars(MACHINES, MONTHS,
                      name='maint',
                      vtype=gp.GRB.INTEGER)


# set upper and lower bounds of "store" variables
for i in PRODS:
    store[i,'INITIAL'].setAttr('ub', 0.0)
    store[i,'INITIAL'].setAttr('lb', 0.0)
    store[i,'JUN'].setAttr('ub', 50.0)
    store[i,'JUN'].setAttr('lb', 50.0)
    
# objective function
model.setObjective(gp.quicksum(contr_profit[i]*sell[i,t] - storage_cost*store[i,t] for i in PRODS for t in MONTHS),
                    GRB.MAXIMIZE)

# add constraints
model.addConstrs((sell[i,t] <= mark_lim[i][t] for i in PRODS for t in MONTHS),
                  name='market_lim')
model.addConstrs((store[i,t] <= 100 for i in PRODS for t in MONTHS),
                  name='storage_lim')
model.addConstrs((make[i,t] + store[i,PREV_MONTH[t]] - sell[i,t] - store[i,t] == 0 for i in PRODS for t in MONTHS),
                  name='storage_link')
model.addConstrs((gp.quicksum(req_proc[i][m]*make[i,t] for i in PRODS) <= 
                  n_of_shifts*shift_length*days_per_month*(exist_mac[m] - maint[m,t])
                  for t in MONTHS for m in MACHINES),
                  name='mac_lim')
model.addConstrs((gp.quicksum(maint[m,t] for t in MONTHS) == req_maint[m] for m in MACHINES))

model.update()

Using license file C:\Users\Nara\gurobi.lic
Academic license - for non-commercial use only - expires 2021-03-12


In [5]:
model.write('4 Factory Planning II.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 161 rows, 163 columns and 432 nonzeros
Model fingerprint: 0x01868e5b
Variable types: 133 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+02]
  Objective range  [5e-01, 1e+01]
  Bounds range     [5e+01, 5e+01]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -175.0000000
Presolve removed 99 rows and 34 columns
Presolve time: 0.00s
Presolved: 62 rows, 129 columns, 278 nonzeros
Variable types: 105 continuous, 24 integer (12 binary)

Root relaxation: objective 1.164550e+05, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 116455.000    0   11 -175.00000 116455.000      -     -    0s
H    0     0                    92755.000000 116455.000 

In [6]:
print('Profit of ${:.2f}'.format(model.objVal))

sell_solution = pd.DataFrame([], columns=PRODS, index=MONTHS)
sell_solution = sell_solution.fillna(0)
for i, t in sell.keys():
    sell_solution[i][t] = sell[i,t].x
print('===SELLING PLAN')
print(sell_solution)

make_solution = pd.DataFrame([], columns=PRODS, index=MONTHS)
make_solution = make_solution.fillna(0)
for i, t in make.keys():
    make_solution[i][t] = make[i,t].x
print('===PRODUCE PLAN')
print(make_solution)

store_solution = pd.DataFrame([], columns=PRODS, index=['INITIAL']+MONTHS)
store_solution = store_solution.fillna(0)
for i, t in store.keys():
    store_solution[i][t] = store[i,t].x
print('===STORAGE PLAN')
print(store_solution[1:])

maint_solution = pd.DataFrame([], columns=MACHINES, index=MONTHS)
maint_solution = maint_solution.fillna(0)
for m, t in maint.keys():
    maint_solution[m][t] = maint[m,t].x
print('===MAINTENANCE PLAN')
print(maint_solution)

Profit of $108855.00
===SELLING PLAN
     PROD1  PROD2  PROD3  PROD4  PROD5  PROD6  PROD7
JAN    500   1000    300    300    800    200    100
FEB    600    500    200      0    400    300    150
MAR    300    600      0      0    500    400    100
APR    100    100    100    100    100      0    100
MAY      0    100    500    100   1000    300      0
JUN    500    500    100    300   1100    500     60
===PRODUCE PLAN
     PROD1  PROD2  PROD3  PROD4  PROD5  PROD6  PROD7
JAN    500   1000    300    300    800    200    100
FEB    600    500    200      0    400    300    150
MAR    400    700    100    100    600    400    200
APR      0      0      0      0      0      0      0
MAY      0    100    500    100   1000    300      0
JUN    550    550    150    350   1150    550    110
===STORAGE PLAN
     PROD1  PROD2  PROD3  PROD4  PROD5  PROD6  PROD7
JAN      0      0      0      0      0      0      0
FEB      0      0      0      0      0      0      0
MAR    100    100    100    10